# Homework - Sprint 04 - Data Transformation

# NYC Flights 2013 Analysis

In [ ]:
install.packages("nycflights13")

library(tidyverse)
library(nycflights13)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
Warning message:
“Failed to locate timezone database”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



In [ ]:

flights <- nycflights13::flights
airlines <- nycflights13::airlines
airports <- nycflights13::airports
planes <- nycflights13::planes
weather <- nycflights13::weather
flights2 <- read_csv("nycflights13_flights.csv", show_col_types = FALSE)

In [ ]:
glimpse(flights)        # glimpse data
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(planes)
cat("\n")
glimpse(weather)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

## Q1: Which airlines had the least departure delay at the beginning of 2013?

In [ ]:
flights %>%
    filter(year == 2013, month == 1, day == 1) %>%      # filter date
    select(carrier, dep_delay) %>%
    arrange(dep_delay) %>%                              # sort by dep_delay in ascending order
    head(5) %>%                                         # top 5 airlines with the least departure delay
    inner_join(airlines, by = "carrier")                # join airlines table to get a full name of each airline

carrier,dep_delay,name
<chr>,<dbl>,<chr>
MQ,-15,Envoy Air
AA,-15,American Airlines Inc.
F9,-14,Frontier Airlines Inc.
EV,-13,ExpressJet Airlines Inc.
B6,-12,JetBlue Airways


## Q2: Which destinations did people travel to the most during the summer (July, August, and September) of 2013?

In [ ]:
flights %>%
    filter(!is.na(dep_time), year == 2013, month %in% c(7,8,9)) %>%     # filter date
    select(year, month, dest) %>%
    group_by(year, month, dest) %>%                                     # group by year, month, and destinations
    summarise(n_filghts = n()) %>%                                      # count the number of flights
    arrange(desc(n_filghts)) %>%                                        # sort by n_flights in descending order 
    head(5) %>%                                                         # top 5 destinations people traveled to the most
    inner_join(airports %>%                                             # join airports table to get full names of destinations
                    select(faa, name), 
               by = c("dest" = "faa"))

`summarise()` has grouped output by 'year', 'month'. You can override using the `.groups` argument.



year,month,dest,n_filghts,name
<int>,<int>,<chr>,<int>,<chr>
2013,8,ORD,1571,Chicago Ohare Intl
2013,9,ORD,1517,Chicago Ohare Intl
2013,8,LAX,1502,Los Angeles Intl
2013,7,ORD,1499,Chicago Ohare Intl
2013,7,LAX,1497,Los Angeles Intl


## Q3: Which airlines had the most canceled flights in 2013?

In [10]:
na_deptime <- flights %>%                                   # a data frame showing the number of canceled flights
  filter(is.na(dep_time)) %>%                               # filter only canceled flights
  group_by(carrier) %>%                                     # group by carrier
  summarise(canceled_flights = n())                         # count the number of canceled flights

flights %>% 
  inner_join(airlines, by = "carrier") %>%                  # join flights and airlines tables
  group_by(carrier, name) %>% 
  select(carrier, name) %>%                                 
  summarise(scheduled_flights = n()) %>%                    # count the number of scheduled flights
  inner_join(na_deptime, by = "carrier") %>%                # join na_deptime table above
  mutate(canceled_flights_proportion =                      # calculate the canceled flights proportion
      round((canceled_flights/scheduled_flights), 4)) %>% 
  arrange(desc(canceled_flights_proportion)) %>%            # sort by the canceled flights proportion in descending order
  head(5)                                                   # top 5 airlines with the most canceled flights

`summarise()` has grouped output by 'carrier'. You can override using the `.groups` argument.



carrier,name,scheduled_flights,canceled_flights,canceled_flights_proportion
<chr>,<chr>,<int>,<int>,<dbl>
OO,SkyWest Airlines Inc.,32,3,0.0938
YV,Mesa Airlines Inc.,601,56,0.0932
9E,Endeavor Air Inc.,18460,1044,0.0566
EV,ExpressJet Airlines Inc.,54173,2817,0.0520
MQ,Envoy Air,26397,1234,0.0467


## Q4: How much was the total traveled distance of each airline each month in 2013?

In [11]:
flights %>%
    filter(!is.na(dep_time)) %>%                                        # filter out all canceled flights
    inner_join(airlines, by = "carrier") %>%                            # join airlines table
    group_by(month, carrier, name) %>%                                  # group by month, carrier, and name 
    summarise(n_flights = n(),                                          # summarise the number of flights and total distance
              total_distance_in_miles = round(sum(distance), 2)) %>%
    arrange(carrier)                                                    # sort by carrier in ascending order

`summarise()` has grouped output by 'month', 'carrier'. You can override using the `.groups` argument.



month,carrier,name,n_flights,total_distance_in_miles
<int>,<chr>,<chr>,<int>,<dbl>
1,9E,Endeavor Air Inc.,1498,717534
2,9E,Endeavor Air Inc.,1353,637366
3,9E,Endeavor Air Inc.,1514,723266
4,9E,Endeavor Air Inc.,1407,691754
5,9E,Endeavor Air Inc.,1388,701809
6,9E,Endeavor Air Inc.,1276,677990
7,9E,Endeavor Air Inc.,1364,736838
8,9E,Endeavor Air Inc.,1378,748201
9,9E,Endeavor Air Inc.,1477,858655


## Q5: What are the fastest flights compared to the distance traveled?

In [12]:
flights %>% 
  select(carrier, air_time, distance) %>%                   
  mutate(distance_interval = case_when(                     # use case-when to segment flights by distance/flight
    distance <= 500 ~ "less than 500 miles",
    distance <= 1000 ~ "500 - 1000 miles",
    distance <= 1500 ~ "1000 - 1500 miles",
    TRUE ~ "more than 1500 miles"
  )) %>% 
  group_by(carrier, distance_interval) %>%                  # group by carrier and distance interval
  summarise(min_air_time = min(air_time, na.rm = T)) %>%    # the lowest airtime of each airline by distance interval
  arrange(carrier, min_air_time)                            # sort by the carrier and the lowest airtime in ascending order

`summarise()` has grouped output by 'carrier'. You can override using the `.groups` argument.



carrier,distance_interval,min_air_time
<chr>,<chr>,<dbl>
9E,less than 500 miles,21
9E,500 - 1000 miles,72
9E,1000 - 1500 miles,127
9E,more than 1500 miles,209
AA,less than 500 miles,29
AA,500 - 1000 miles,94
AA,1000 - 1500 miles,125
AA,more than 1500 miles,173
AS,more than 1500 miles,277
